In [1]:
import pandas as pd

In [12]:
df = pd.DataFrame([], columns=["year", "team", "rank"])
for year in range(1991, 2024, 4):
    df_temp = pd.read_csv("data/matches_" + str(year) + ".csv")
    df_new = pd.DataFrame(sorted(df_temp["home"].dropna().unique()), columns=["team"])
    df_new["year"] = year
    df_new["rank"] = ""
    df = df.append(df_new)

df.to_csv("data/ranks.csv", index=False)